In [1]:
__author__ = 'Monika Soraisam'
__email__ = 'monika.soraisam@noirlab.edu'

# This notebook shows how one can obtain all the primitives for a given observation type and the arguments associated with that primitive

In [2]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import inspect

# for bokeh apps to load from the jupyter notebook on a different browser tab
import nest_asyncio
nest_asyncio.apply()

In [48]:
# Now import the DRAGONS libraries 
import astrodata
import gemini_instruments

from gempy.scripts import showpars
from gempy.utils.showrecipes import showrecipes
from gempy.utils.showrecipes import showprims

from recipe_system.mappers.primitiveMapper import PrimitiveMapper
from recipe_system.mappers.recipeMapper import RecipeMapper

import textwrap

In [4]:
from recipe_system.reduction.coreReduce import Reduce
from recipe_system import cal_service
from gempy.adlibrary import dataselect
from gempy.utils import logutils

In [5]:
## prep the reduction folder
def prep_reduction_folder(data_root, obsid):
    reduction_path = Path(f"{data_root}/{obsid}/reduction")

    if not reduction_path.exists():
        os.mkdir(reduction_path.as_posix())
        print (f"directory for dragons reduction created")
    
    ## change the cwd to the reduction folder
    os.chdir(reduction_path.as_posix())
    print(f"Current working directory is: {os.getcwd()}")

    ## write the configuration file 
    mydb = "dragons_for_goats.db" 
    mydb_path = reduction_path.as_posix() + '/' + mydb
    print (mydb_path)
    
    dragons_rc = reduction_path.as_posix() + '/dragonsrc'
    print (dragons_rc)
    
    with open(dragons_rc, "w") as f:
        f.write("[calibs]\ndatabases = {0} get store".format(mydb_path))

    return dragons_rc, mydb_path


# GMOS longslit example

In [6]:
#data_path = "/data/goats_dev_data/example_data/data/ZTF18acppavh/GEM" ## linux machine 
data_path = "/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18aabfthf/GEM" ## mac 
obsid = 'GS-2021A-DD-102-9'
dragons_rc, mydb_path = prep_reduction_folder(data_path, obsid)

Current working directory is: /Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18aabfthf/GEM/GS-2021A-DD-102-9/reduction
/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18aabfthf/GEM/GS-2021A-DD-102-9/reduction/dragons_for_goats.db
/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18aabfthf/GEM/GS-2021A-DD-102-9/reduction/dragonsrc


In [7]:
# initialize the calibration database and complete the set-up 
caldb = cal_service.LocalDB(mydb_path, force_init=True) # 


In [8]:
logutils.config(file_name='gmos_data_reduction.log') # logger

### Generate filelist for the data reduction

In [9]:
def generate_filelists(location, obsid):
    """
    Parameters
    ----------
    location: str
        Root folder where the Gemini data for a given target is located
    obsid: str
        Gemini observation ID 
    """

    
    all_files = [str(pp) for pp in list(Path(location+"/"+obsid).glob('*.fits'))]
    all_files.sort()
    print (f'The total number of files for observation ID {obsid} is {len(all_files)}')

    cal_file_tags = ['BIAS','DARK','FLAT','ARC','PINHOLE','RONCHI','FRINGE'] #fetched from Obs Type search field on GOA, which is relevant for DRAGONS

    meta_keys = cal_file_tags + ['BPM','standard','object','unknown']
    all_meta = {}

    for K in meta_keys:
        all_meta[K] = {'file':[],
                        'group_id':[],
                        'exp':[],
                        'objname':[],
                        'wave':[],
                        'waveband':[],
                        'date':[],
                        'roi':[],
                        }

    object_files = []
    for i,F in enumerate(all_files):
        ad = astrodata.open(F)

        K = "unknown"
        if "BPM" in ad.tags:
            K = "BPM"
        elif "PREPARED" in ad.tags:
            continue
        elif ("STANDARD" in ad.tags or ad.observation_class()=="partnerCal" or ad.observation_class()=="progCal") and ("UNPREPARED" in ad.tags) and (ad.observation_type()=="OBJECT"):
            K = "standard"            
        elif "CAL" in ad.tags and "UNPREPARED" in ad.tags:
            for tag in cal_file_tags:
                if tag in ad.tags:
                    K = tag
        elif ad.observation_class()=="science" and "UNPREPARED" in ad.tags:
            K = "object"
            
        all_meta[K]['file'].append(F)
        # group_id seems to be not implemented for GNIRS yet
        if "GNIRS" in ad.instrument():
            all_meta[K]['group_id'].append(None)
        else:
            all_meta[K]['group_id'].append(ad.group_id())
        all_meta[K]['exp'].append(ad.exposure_time())
        all_meta[K]['objname'].append(ad.object())
        all_meta[K]['wave'].append(ad.central_wavelength())
        all_meta[K]['waveband'].append(ad.wavelength_band())
        all_meta[K]['date'].append(ad.ut_date())
        all_meta[K]['roi'].append(ad.detector_roi_setting()) 
        #print (F.split('/')[-1], ad.object(), ad.tags)
    
    return all_meta


In [10]:
all_meta = generate_filelists(data_path, obsid)

for K,V in all_meta.items():
    if len(V['file'])==0:
        continue
    print (f"There are {len(V['file'])} files for observation type {K}")


The total number of files for observation ID GS-2021A-DD-102-9 is 58
There are 50 files for observation type BIAS
There are 2 files for observation type FLAT
There are 2 files for observation type ARC
There are 3 files for observation type object
There are 1 files for observation type unknown


In [11]:
DF_bias = pd.DataFrame(all_meta['BIAS'])
DF_flat = pd.DataFrame(all_meta['FLAT'])
#DF_bpm = pd.DataFrame(all_meta['BPM'])
DF_arc = pd.DataFrame(all_meta['ARC'])
DF_object = pd.DataFrame(all_meta['object'])


In [12]:
## add BPM to the calibration database (the BPM is already processed)
for F in caldb.list_files():
    print (F)

# for F in DF_bpm['file'].values:
#     caldb.add_cal(F)

for F in caldb.list_files():
    print (F)

# Let's consider the observation type Flat


In [20]:
ad = astrodata.open(DF_flat['file'].values[0]) ## choose any file from this observation type, here I chose the first file
tags = ad.tags
instpkg = ad.instrument(generic=True).lower()
print (f"Instrument package is {instpkg} and tags are {tags}")

Instrument package is gmos and tags are {'SIDEREAL', 'UNPREPARED', 'GEMINI', 'GMOS', 'CAL', 'RAW', 'SPECT', 'LS', 'SOUTH', 'GCALFLAT', 'FLAT'}


In [23]:
# instantiate the PrimitiveMapper class
pmapper = PrimitiveMapper(tags, instpkg)

# get the primitive "class" appropriate for the given instrument and tags
pclass = pmapper.get_applicable_primitives() 

In [24]:
pclass?

Init signature: pclass(adinputs, **kwargs)
Docstring:      "Magic" class to provide the correct class for N&S and classic data
File:           /opt/anaconda3/envs/goats-env/lib/python3.10/site-packages/geminidr/gmos/primitives_gmos_longslit.py
Type:           type
Subclasses:     

In [26]:
# initialize pclass with the list of astrodata of the observation type above to retrieve all the relevant primitives  
ad_input_list = []
for F in DF_flat['file'].values:
    ad_input_list.append(astrodata.open(F))

p = pclass(ad_input_list, config_file=dragons_rc)

# now show the primitive and its associated arguments
for item in dir(p):
   if not item.startswith('_') and inspect.ismethod(getattr(p, item)):
       print (item)
       print (showpars.showpars(p, item, tags, show_docstring=True)) ## note that this will crash if the primitive doesn't have a docstring
       break

ADUToElectrons
Dataset tagged as {'SIDEREAL', 'UNPREPARED', 'GEMINI', 'GMOS', 'CAL', 'RAW', 'SPECT', 'LS', 'SOUTH', 'GCALFLAT', 'FLAT'}

Settable parameters on 'ADUToElectrons':
Name                 Current setting      Description

suffix               '_ADUToElectrons'    Filename suffix

Docstring for 'ADUToElectrons':

This primitive will convert the units of the pixel data extensions
of the input AstroData object from ADU to electrons by multiplying
by the gain. The gain keyword in each extension is then set to 1.0
to represent the new conversion factor.

Parameters
----------
suffix: str/None
    suffix to be added to output filenames

None


## Let's inspect the code for showpars to adapt it for our needs

In [40]:
showpars.showpars??

Signature: showpars.showpars(pobj, primname, tags, show_docstring)
Docstring: <no docstring>
Source:   
def showpars(pobj, primname, tags, show_docstring):
    print(f"Dataset tagged as {tags}\n")
    if primname not in pobj.params:
        raise KeyError(f"{primname} doesn't exist for "
                       "this data type.")

    print(f"Settable parameters on '{primname}':")
    print("=" * 40)
    print(f"{'Name':20s} {'Current setting':20s} Description\n")

    params = pobj.params[primname]
    for k, v in params.items():
        if not k.startswith("debug"):
            print(f"{k:20s} {v!r:20s} {params.doc(k)}")

    if show_docstring:
        print(f"\nDocstring for '{primname}':")
        print("=" * 40)
        print(textwrap.dedent(getattr(pobj, primname).__doc__))
File:      /opt/anaconda3/envs/goats-env/lib/python3.10/site-packages/gempy/scripts/showpars.py
Type:      function

In [53]:
## let's wrap the showpars part in try...except... t avoid the crash and trim off the docstring, if it exists, before the Parameters portion
for item in dir(p):
   if not item.startswith('_') and inspect.ismethod(getattr(p, item)):
       print (item)

       print(f"Settable parameters on '{item}':")
       print("=" * 40)
       print(f"{'Name':20s} {'Current setting':20s} Description\n")
       params = p.params[item]
       for k, v in params.items():
           if not k.startswith("debug"):
               print(f"{k:20s} {v!r:20s} {params.doc(k)}")
       try:
           print(f"\nDocstring for '{item}':")
           print("=" * 40)
           X = (getattr(p, item).__doc__)
           print(textwrap.dedent(X.split('Parameters')[0]))
       except Exception as e:
           pass

       break

ADUToElectrons
Settable parameters on 'ADUToElectrons':
Name                 Current setting      Description

suffix               '_ADUToElectrons'    Filename suffix

Docstring for 'ADUToElectrons':

This primitive will convert the units of the pixel data extensions
of the input AstroData object from ADU to electrons by multiplying
by the gain. The gain keyword in each extension is then set to 1.0
to represent the new conversion factor.


